# Preparing FIA Tree Measurements
We will be utilizing re-measured FIA forest inventory plots to fit equations employed by the Forest Vegetation Simulator to model forest growth and yield. In recent years, the distribution of the FIA database for each US state has included tables for Forest Vegetation Simulator (FVS)-ready input data ([see official documentation for using FIA data in FVS here](https://www.fs.fed.us/fvs/documents/FIA_Data_Quick_Start_Guide_20200914.pdf)). These data can be imported directly into FVS and used to simulate forest growth and yield. Some, but not all, of the FIA plots in the database have been remeasured. These remeasured plots will provide the basis for refitting some of the fundamental equations employed by FVS.

Of potential interest, the FVS-ready FIA data is provided at several different levels of aggregation: 
* **Plot** - All trees across an FIA plot are considered to occur within a single FVS stand. This may mix together trees occuring on different conditions if there are forest type or landcover breaks within the footprint of the FIA plot. These data are in the FVS_STANDINIT_PLOT table in the FIA database.
* **Condition** - All trees that occur within a single condition on an FIA plot are considered to occur within a single FVS stand. These data are in the FVS_STANDINIT_COND table in the FIA database.
* **Subplot** - All trees that occur within a single subplot on an FIA plot are considered to occur within a single FVS stand. This may mix together trees occuring on different conditions if there are forest type or landcover breaks within the footprint of the FIA plot. This approach would be comparable to "gap-scale" modeling. These data are in the FVS_PLOTINIT_PLOT table in the FIA database.

In this project, we focus on utilizing the **condition-level** aggregation of FIA measurements. This is the most consistent with the original intended use of FVS for modeling stand-level behavior. 

## Using FVS-Compiled Inventory
FVS growth and mortality equations utilize features (i.e., covariates) that are complex to estimate because they require additional species-specific equations that may vary by region (e.g., for imputing missing crown ratios, tree heights, or calculating plot-level crown competition factor). Instead of calculating these variables directly from the FIA inventory data, we utilize the FVS model to compile the inventory at the time of measurement, generating all the variables that will be employed in fitting growth and yield equations. In essence, FVS is used to translate the input data into another format, augmenting the raw FIA database tables with derived variables we need for refitting the equations. For refitting these fundamental equations, we are not utilizing data generated from the growing-forward of FIA plots, we are only utilizing data at the time field measurements were collected. 

For each unique CASEID in the FVS tables (which represents a single FVS simulation), there should only be a single year of data. The STANDID field for each record can be parsed to uniquely identify FIA conditions to support the joining of measurements from the same condition at different times. 

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy

In [2]:
FVS_DB = '../data/interim/PNW_FIADB_COND_FVSOut.db'
fvs_engine = sqlalchemy.create_engine(f'sqlite:///{FVS_DB}')

In [3]:
SQL = """
SELECT SUBSTR(StandID,1,4) || SUBSTR(StandID,13,11) AS FIACOND,
       TreeId, PtIndex, Year, SpeciesFIA AS Species, SpeciesFVS as FVS_ALPHA, DBH, 
       TPA, Ht, PctCr AS CR, CrWidth AS CW, PtBAL, 
       TreeVal, CaseID, StandID, TreeIndex
FROM FVS_TreeList
ORDER BY FIACOND, TreeId, Year
"""
fvs_trees = pd.read_sql(SQL, fvs_engine)
fvs_trees.columns = [col.upper() for col in fvs_trees.columns]
INT_COLS = ['YEAR', 'TREEID', 'PTINDEX', 'SPECIES', 'TREEVAL', 'CR', 'TREEINDEX']
fvs_trees[INT_COLS] = fvs_trees[INT_COLS].astype(float).astype(int)
fvs_trees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1357949 entries, 0 to 1357948
Data columns (total 16 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   FIACOND    1357949 non-null  object 
 1   TREEID     1357949 non-null  int32  
 2   PTINDEX    1357949 non-null  int32  
 3   YEAR       1357949 non-null  int32  
 4   SPECIES    1357949 non-null  int32  
 5   FVS_ALPHA  1357949 non-null  object 
 6   DBH        1357949 non-null  float64
 7   TPA        1357949 non-null  float64
 8   HT         1357949 non-null  float64
 9   CR         1357949 non-null  int32  
 10  CW         1357949 non-null  float64
 11  PTBAL      1357949 non-null  float64
 12  TREEVAL    1357949 non-null  int32  
 13  CASEID     1357949 non-null  object 
 14  STANDID    1357949 non-null  object 
 15  TREEINDEX  1357949 non-null  int32  
dtypes: float64(5), int32(7), object(4)
memory usage: 129.5+ MB


### Parse `STANDID` to uniquely identify each FIA condition. 
According to the [documentation](https://www.fs.fed.us/fvs/documents/FIA_Data_Quick_Start_Guide_20200914.pdf), `STANDID` in the `FVS_STANDINIT_COND` table is a string concatenation of: 

STATECD(4) + INVYR(4) + CYCLE(2) + SUBCYCLE(2) + UNITCD(2) + COUNTYCD(3) + PLOT(5) + CONDID (1)

To uniquely identify a single condition across inventories, then, we can extract the STATECD, UNITCD, COUNTYCD, PLOT, and CONDID as a new attribute.

In [4]:
SQL = """
SELECT SUBSTR(StandID,1,4) || SUBSTR(StandID,13,11) AS FIACOND, StandID,
       Year, BA, TopHt, CCF as STAND_CCF,
       CaseID
FROM FVS_Summary2
ORDER BY FIACOND, Year
"""
fvs_stands = pd.read_sql(SQL, fvs_engine)
fvs_stands.columns = [col.upper() for col in fvs_stands.columns]
fvs_stands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88156 entries, 0 to 88155
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   FIACOND    88156 non-null  object 
 1   STANDID    88156 non-null  object 
 2   YEAR       88156 non-null  int64  
 3   BA         88156 non-null  float64
 4   TOPHT      88156 non-null  int64  
 5   STAND_CCF  88156 non-null  int64  
 6   CASEID     88156 non-null  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 4.7+ MB


In [5]:
OR_FIADB = '../data/raw/FIADB_OR.db'
or_engine = sqlalchemy.create_engine(f'sqlite:///{OR_FIADB}')

WA_FIADB = '../data/raw/FIADB_WA.db'
wa_engine = sqlalchemy.create_engine(f'sqlite:///{WA_FIADB}')

CA_FIADB = '../data/raw/FIADB_CA.db'
ca_engine = sqlalchemy.create_engine(f'sqlite:///{CA_FIADB}')

SQL = """
SELECT SUBSTR(STAND_ID,1,4) || SUBSTR(STAND_ID,13,11) AS FIACOND,
       INV_YEAR AS YEAR,
       LATITUDE AS LAT, LONGITUDE AS LON, REGION, LOCATION,
       ASPECT, SLOPE, ELEVFT,
       SITE_SPECIES, SITE_INDEX, SITE_INDEX_BASE_AG, 
       VARIANT
FROM FVS_STANDINIT_COND
ORDER BY FIACOND, YEAR
"""

engines = [or_engine, wa_engine, ca_engine]
fia_stands = pd.concat([pd.read_sql(SQL, engine) for engine in engines], 
                       axis=0).dropna(how='any').reset_index(drop=True)

INT_COLS = ['YEAR', 'REGION', 'LOCATION', 'SITE_INDEX', 'SITE_SPECIES',
            'SITE_INDEX_BASE_AG', 'ELEVFT', 'SLOPE', 'ASPECT']
fia_stands[INT_COLS] = fia_stands[INT_COLS].astype(float).astype(int)

fia_stands = fia_stands.drop_duplicates(subset=['FIACOND']).drop(['YEAR'], axis=1)

fia_stands.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22977 entries, 0 to 37129
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   FIACOND             22977 non-null  object 
 1   LAT                 22977 non-null  float64
 2   LON                 22977 non-null  float64
 3   REGION              22977 non-null  int32  
 4   LOCATION            22977 non-null  int32  
 5   ASPECT              22977 non-null  int32  
 6   SLOPE               22977 non-null  int32  
 7   ELEVFT              22977 non-null  int32  
 8   SITE_SPECIES        22977 non-null  int32  
 9   SITE_INDEX          22977 non-null  int32  
 10  SITE_INDEX_BASE_AG  22977 non-null  int32  
 11  VARIANT             22977 non-null  object 
dtypes: float64(2), int32(8), object(2)
memory usage: 1.6+ MB


In [6]:
SQL = """
SELECT SUBSTR(STAND_ID,1,4) || SUBSTR(STAND_ID,13,11) AS FIACOND, FIRST_YEAR, REMEAS_YEAR
FROM FVS_STANDINIT_COND
INNER JOIN 
    (SELECT SUBSTR(STAND_ID,1,4) || SUBSTR(STAND_ID,13,11) AS FIACOND1, 
    COUNT(STAND_ID) AS MEAS_COUNT, MIN(INV_YEAR) AS FIRST_YEAR, MAX(INV_YEAR) AS REMEAS_YEAR
    FROM FVS_STANDINIT_COND
    GROUP BY FIACOND1
    HAVING MEAS_COUNT > 1)
ON FIACOND = FIACOND1 AND FVS_STANDINIT_COND.INV_YEAR = REMEAS_YEAR
INNER JOIN COND ON FVS_STANDINIT_COND.Stand_CN = COND.CN
WHERE (COND.DSTRBCD1 = 0 OR COND.DSTRBCD1 IS NULL) AND (COND.DSTRBCD2 = 0 OR COND.DSTRBCD2 IS NULL) AND (COND.DSTRBCD3 = 0 OR COND.DSTRBCD3 IS NULL)
"""

engines = [or_engine, wa_engine, ca_engine]
good_conds = pd.concat([pd.read_sql(SQL, engine) for engine in engines], 
                       axis=0).dropna(how='any').reset_index(drop=True)
INT_COLS = ['FIRST_YEAR', 'REMEAS_YEAR']
good_conds[INT_COLS] = good_conds[INT_COLS].astype(int)
good_conds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36423 entries, 0 to 36422
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   FIACOND      36423 non-null  object
 1   FIRST_YEAR   36423 non-null  int32 
 2   REMEAS_YEAR  36423 non-null  int32 
dtypes: int32(2), object(1)
memory usage: 569.2+ KB


In [7]:
SQL = """
SELECT SUBSTR(FVS_STANDINIT_COND.STAND_ID,1,4) || SUBSTR(FVS_STANDINIT_COND.STAND_ID,13,11) AS FIACOND, 
       FVS_STANDINIT_COND.STAND_ID AS STANDID, INV_YEAR AS YEAR, TREE_CN AS TREECN, PREV_TRE_CN AS PREV_TREECN, 
       TREE_ID AS TREEID, SPECIES, 
       TREE_COUNT *1000 AS TPA1000, DIAMETER *1000 AS DBH1000, FVS_TREEINIT_COND.HT, CRRATIO AS CR, HISTORY AS TREEVAL
FROM FVS_TREEINIT_COND
LEFT JOIN TREE ON TREE_CN = TREE.CN
JOIN FVS_STANDINIT_COND ON FVS_TREEINIT_COND.STAND_ID = FVS_STANDINIT_COND.STAND_ID
ORDER BY STANDID, TREECN
"""

engines = [or_engine, wa_engine, ca_engine]
fia_trees = pd.concat([pd.read_sql(SQL, engine) for engine in engines], 
                      axis=0).dropna(subset=['SPECIES', 'TPA1000', 'DBH1000'], how='any')

INT_COLS = ['TREEID', 'YEAR', 'SPECIES', 'TREEVAL', 'DBH1000', 'TPA1000']
fia_trees[INT_COLS] = fia_trees[INT_COLS].astype(float).round(0).astype(int)
fia_trees[['CR', 'HT']] = fia_trees[['CR', 'HT']].round(0)
fia_trees.loc[fia_trees.TREEVAL > 1, 'TPA1000'] = 0

fia_trees.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1528036 entries, 0 to 393690
Data columns (total 12 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   FIACOND      1528036 non-null  object 
 1   STANDID      1528036 non-null  object 
 2   YEAR         1528036 non-null  int32  
 3   TREECN       1528036 non-null  object 
 4   PREV_TREECN  569330 non-null   object 
 5   TREEID       1528036 non-null  int32  
 6   SPECIES      1528036 non-null  int32  
 7   TPA1000      1528036 non-null  int32  
 8   DBH1000      1528036 non-null  int32  
 9   HT           1381249 non-null  float64
 10  CR           1199249 non-null  float64
 11  TREEVAL      1528036 non-null  int32  
dtypes: float64(2), int32(6), object(4)
memory usage: 116.6+ MB


In [8]:
fvs_trees = fvs_trees.loc[fvs_trees.FIACOND.isin(good_conds.FIACOND)]
fia_trees = fia_trees.loc[fia_trees.FIACOND.isin(good_conds.FIACOND)]

In [9]:
fvs_trees['TPA1000'] = (fvs_trees.TPA * 1000).round(0).astype(int)
fvs_trees['DBH1000'] = (fvs_trees.DBH * 1000).round(0).astype(int)

MERGE_COLS = ['STANDID', 'TREEID', 'TPA1000', 'DBH1000']

single_fvs_idx = fvs_trees.groupby(by=MERGE_COLS)['DBH'].count().loc[fvs_trees.groupby(by=MERGE_COLS)['DBH'].count() == 1].index
single_fia_idx = fia_trees.groupby(by=MERGE_COLS)['TREECN'].count().loc[fia_trees.groupby(by=MERGE_COLS)['TREECN'].count() == 1].index
use_fia_idx = [x for x in single_fia_idx if x in single_fvs_idx]

fvs_trees['TREECN'] = np.nan
fvs_trees['PREV_TREECN'] = np.nan
fvs_trees = fvs_trees.set_index(MERGE_COLS)
fvs_trees.loc[use_fia_idx, ['TREECN', 'PREV_TREECN']] = fia_trees.set_index(MERGE_COLS).loc[use_fia_idx, ['TREECN', 'PREV_TREECN']]

fvs_trees = fvs_trees.reset_index()
fvs_trees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818884 entries, 0 to 818883
Data columns (total 20 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   STANDID      818884 non-null  object 
 1   TREEID       818884 non-null  int64  
 2   TPA1000      818884 non-null  int64  
 3   DBH1000      818884 non-null  int64  
 4   FIACOND      818884 non-null  object 
 5   PTINDEX      818884 non-null  int32  
 6   YEAR         818884 non-null  int32  
 7   SPECIES      818884 non-null  int32  
 8   FVS_ALPHA    818884 non-null  object 
 9   DBH          818884 non-null  float64
 10  TPA          818884 non-null  float64
 11  HT           818884 non-null  float64
 12  CR           818884 non-null  int32  
 13  CW           818884 non-null  float64
 14  PTBAL        818884 non-null  float64
 15  TREEVAL      818884 non-null  int32  
 16  CASEID       818884 non-null  object 
 17  TREEINDEX    818884 non-null  int32  
 18  TREECN       817805 non-

In [10]:
for idx, row in fvs_trees.loc[pd.isnull(fvs_trees.TREECN)].iterrows():
    match = fia_trees.loc[(fia_trees.STANDID == row['STANDID'])&(fia_trees.TREEID == row['TREEID']), ['TREECN', 'PREV_TREECN']]
    if len(match) == 1:
        fvs_trees.loc[idx, ['TREECN', 'PREV_TREECN']] = match.values[0]
fvs_trees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818884 entries, 0 to 818883
Data columns (total 20 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   STANDID      818884 non-null  object 
 1   TREEID       818884 non-null  int64  
 2   TPA1000      818884 non-null  int64  
 3   DBH1000      818884 non-null  int64  
 4   FIACOND      818884 non-null  object 
 5   PTINDEX      818884 non-null  int32  
 6   YEAR         818884 non-null  int32  
 7   SPECIES      818884 non-null  int32  
 8   FVS_ALPHA    818884 non-null  object 
 9   DBH          818884 non-null  float64
 10  TPA          818884 non-null  float64
 11  HT           818884 non-null  float64
 12  CR           818884 non-null  int32  
 13  CW           818884 non-null  float64
 14  PTBAL        818884 non-null  float64
 15  TREEVAL      818884 non-null  int32  
 16  CASEID       818884 non-null  object 
 17  TREEINDEX    818884 non-null  int32  
 18  TREECN       818244 non-

In [11]:
for idx, row in fvs_trees.loc[pd.isnull(fvs_trees.TREECN)].iterrows():
    match = fia_trees.loc[
        (fia_trees.STANDID == row['STANDID'])&\
        (fia_trees.TREEID == row['TREEID'])&\
        (fia_trees.DBH1000 == row['DBH1000'])&\
        (fia_trees.TPA1000 == row['TPA1000'])&\
        (fia_trees.CR == row['CR'])&\
        (fia_trees.HT == row['HT']), 
         ['TREECN', 'PREV_TREECN']]
    if len(match) == 1:
        fvs_trees.loc[idx, ['TREECN', 'PREV_TREECN']] = match.values[0]
fvs_trees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818884 entries, 0 to 818883
Data columns (total 20 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   STANDID      818884 non-null  object 
 1   TREEID       818884 non-null  int64  
 2   TPA1000      818884 non-null  int64  
 3   DBH1000      818884 non-null  int64  
 4   FIACOND      818884 non-null  object 
 5   PTINDEX      818884 non-null  int32  
 6   YEAR         818884 non-null  int32  
 7   SPECIES      818884 non-null  int32  
 8   FVS_ALPHA    818884 non-null  object 
 9   DBH          818884 non-null  float64
 10  TPA          818884 non-null  float64
 11  HT           818884 non-null  float64
 12  CR           818884 non-null  int32  
 13  CW           818884 non-null  float64
 14  PTBAL        818884 non-null  float64
 15  TREEVAL      818884 non-null  int32  
 16  CASEID       818884 non-null  object 
 17  TREEINDEX    818884 non-null  int32  
 18  TREECN       818844 non-

In [12]:
# tmp = (fvs_trees.loc[pd.isnull(trees.TREECN)]
#        .drop(['TREECN', 'PREV_TREECN'], axis=1)
#        .merge(fia_trees[MERGE_COLS + ['TREECN', 'PREV_TREECN']], 
#               on=MERGE_COLS, how='left'))
# for (standid, treeindex) in tmp.dropna(subset=['TREECN']).groupby(by=['STANDID', 'TREEINDEX']).groups.keys():
#     match = tmp.loc[(tmp.STANDID == standid)&(tmp.TREEINDEX == treeindex)]
#     fvs_trees.loc[(fvs_trees.STANDID == standid)&(fvs_trees.TREEINDEX == treeindex), ['TREECN']] = match.iloc[0]['TREECN']
#     fvs_trees.loc[(fvs_trees.STANDID == standid)&(fvs_trees.TREEINDEX == treeindex), ['PREV_TREECN']] = match.iloc[0]['PREV_TREECN']
#     tmp = tmp.loc[tmp.TREECN != match.iloc[0]['TREECN']]
# fvs_trees.info()

In [13]:
# calculate a relative site index to allow interspecific translation
fia_stands['REL_SITE_INDEX'] = np.nan
site_max = fia_stands.groupby(by=['SITE_SPECIES', 'SITE_INDEX_BASE_AG'])['SITE_INDEX'].max()
for (spp, age) in fia_stands.groupby(by=['SITE_SPECIES', 'SITE_INDEX_BASE_AG']).groups:
    match = (fia_stands.SITE_SPECIES == spp) & (fia_stands.SITE_INDEX_BASE_AG == age)
    fia_stands.loc[match, 'REL_SITE_INDEX'] = fia_stands.loc[match, 'SITE_INDEX'] / site_max.loc[(spp, age)]
fia_stands.sample(10)

,FIACOND,LAT,LON,REGION,LOCATION,ASPECT,SLOPE,ELEVFT,SITE_SPECIES,SITE_INDEX,SITE_INDEX_BASE_AG,VARIANT,REL_SITE_INDEX
6435,004102019585862,43.088874,-122.619821,6,615,178,30,2300,202,120,100,WC,0.538117
33804,000603017834541,38.762116,-120.747923,5,503,290,10,2100,122,54,50,WS,0.490909
36777,000605109645061,38.273420,-120.117108,5,516,194,20,6300,122,48,50,WS,0.436364
29442,005309019852701,48.395055,-118.215629,6,608,118,8,1500,202,96,50,EC,0.516129
22436,005306027768412,47.282603,-123.895756,6,609,0,3,300,202,96,50,PN,0.516129
2266,004101003924522,44.367848,-123.715156,7,709,63,30,500,351,55,20,PN,0.604396
21536,005305073750991,48.868928,-121.618366,6,605,239,12,2500,11,170,100,WC,0.949721
35534,000605005574811,38.456904,-120.509797,5,503,190,60,2900,202,74,50,WS,0.397849
15471,004104001949152,44.999831,-116.927711,6,619,172,25,3500,122,120,100,BM,0.764331
20563,005305055886682,48.561984,-122.961729,6,609,150,10,100,202,86,50,PN,0.462366


In [14]:
SQL = 'SELECT * FROM REF_SPECIES'
engines = [or_engine, wa_engine, ca_engine]
fia_spp = pd.concat([pd.read_sql(SQL, engine) for engine in engines], 
                axis=0)
fia_spp['SPCD'] = fia_spp['SPCD'].astype(int)
fia_spp = fia_spp.drop_duplicates(subset=['SPCD'])
fia_spp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2677 entries, 0 to 2676
Data columns (total 79 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   SPCD                          2677 non-null   int32  
 1   COMMON_NAME                   2677 non-null   object 
 2   GENUS                         2677 non-null   object 
 3   SPECIES                       2677 non-null   object 
 4   VARIETY                       21 non-null     object 
 5   SUBSPECIES                    8 non-null      object 
 6   SPECIES_SYMBOL                2677 non-null   object 
 7   E_SPGRPCD                     2677 non-null   int64  
 8   W_SPGRPCD                     2677 non-null   int64  
 9   C_SPGRPCD                     879 non-null    float64
 10  P_SPGRPCD                     1343 non-null   float64
 11  MAJOR_SPGRPCD                 2677 non-null   int64  
 12  STOCKING_SPGRPCD              2671 non-null   float64
 13  FOR

### Join with other tables to get stand-level attributes
To get stand-level attributes for the trees (which are used to incorporate competition and environmental effects recorded at the stand- or plot-level), we will then join the tree table with these other tables to fetch those attributes.

In [16]:
fia_stands.loc[fia_stands.FIACOND == '000601015502951']

,FIACOND,LAT,LON,REGION,LOCATION,ASPECT,SLOPE,ELEVFT,SITE_SPECIES,SITE_INDEX,SITE_INDEX_BASE_AG,VARIANT,REL_SITE_INDEX


In [18]:
joined = fvs_trees.merge(fvs_stands,
                         on=['CASEID', 'STANDID', 'FIACOND', 'YEAR'], 
                         how='left')
joined = joined.merge(fia_stands, on='FIACOND', how='inner')  # fia_stands has location info, dump stands without location info still in treelist
joined['RELHT'] = (joined['HT'] / joined['TOPHT']).clip(None,1.5)
joined.head()

,STANDID,TREEID,TPA1000,DBH1000,FIACOND,PTINDEX,YEAR,SPECIES,FVS_ALPHA,DBH,...,LOCATION,ASPECT,SLOPE,ELEVFT,SITE_SPECIES,SITE_INDEX,SITE_INDEX_BASE_AG,VARIANT,REL_SITE_INDEX,RELHT
0,00062004050401015502952,108,131365,2300,000601015502952,1,2004,202,DF,2.3,...,510,188,25,2700,202,67,50,NC,0.360215,0.260000
1,00062004050401015502952,1081003,131365,100,000601015502952,1,2004,81,IC,0.1,...,510,188,25,2700,202,67,50,NC,0.360215,0.020200
2,00062014060401015502952,1081003,245308,100,000601015502952,1,2014,81,IC,0.1,...,510,188,25,2700,202,67,50,NC,0.360215,0.036071
3,00062004050401015502952,109,10546,9400,000601015502952,1,2004,202,DF,9.4,...,510,188,25,2700,202,67,50,NC,0.360215,0.820000
4,00062004050401015502952,110,10546,13700,000601015502952,1,2004,202,DF,13.7,...,510,188,25,2700,202,67,50,NC,0.360215,1.240000


In [19]:
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 739583 entries, 0 to 739582
Data columns (total 36 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   STANDID             739583 non-null  object 
 1   TREEID              739583 non-null  int64  
 2   TPA1000             739583 non-null  int64  
 3   DBH1000             739583 non-null  int64  
 4   FIACOND             739583 non-null  object 
 5   PTINDEX             739583 non-null  int32  
 6   YEAR                739583 non-null  int32  
 7   SPECIES             739583 non-null  int32  
 8   FVS_ALPHA           739583 non-null  object 
 9   DBH                 739583 non-null  float64
 10  TPA                 739583 non-null  float64
 11  HT                  739583 non-null  float64
 12  CR                  739583 non-null  int32  
 13  CW                  739583 non-null  float64
 14  PTBAL               739583 non-null  float64
 15  TREEVAL             739583 non-nul

### Calculate Diameter Inside Bark
The FVS large tree growth equation predicts inside-bark squared diameter increment. We need to calculate DIB for each tree. The documentation for each FVS regional variant provides equations to calculate either DIB or double bark width (which can be subtracted from DBH to get DIB) for each species. We'll need to crosswalk the FIA species code to the FVS species code to use these equations.

In [20]:
BARK_EQS = '../data/raw/fvs_bark_equations.csv'
bark_eqs = pd.read_csv(BARK_EQS).set_index(['VARIANT', 'SPECIES'])
bark_eqs.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 333 entries, ('PN', 'SF') to ('WS', 'OH')
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   PREDICTS  333 non-null    object 
 1   B1        333 non-null    float64
 2   B2        333 non-null    float64
 3   B3        333 non-null    float64
 4   EQN       333 non-null    object 
 5   CITATION  173 non-null    object 
dtypes: float64(3), object(3)
memory usage: 19.4+ KB


In [21]:
bark_eqs.head()

PREDICTS   B1        B2   B3    EQN               CITATION
VARIANT SPECIES                                                           
PN      SF           DIB  0.0  0.904973  1.0  4.2.1  Larsen and Hann, 1985
        WF           DIB  0.0  0.904973  1.0  4.2.1  Larsen and Hann, 1985
        GF           DIB  0.0  0.904973  1.0  4.2.1  Larsen and Hann, 1985
        AF           DIB  0.0  0.904973  1.0  4.2.1  Larsen and Hann, 1985
        RF           DIB  0.0  0.904973  1.0  4.2.1  Larsen and Hann, 1985

In [22]:
def calc_dib(dbh, b1, b2, b3, dib_or_dbt='DIB'):
    """Calculates Diameter Inside Bark from DBH and a 3-coefficient model.
    
    Parameters
    ----------
    dbh : array-like or numeric
      diameter at breast height
    b1, b2, b3 : numeric
      coefficients for the bark ratio equation
    dib_or_dbt : str
      whether the bark ratio equation returns diameter insight bark ('DIB') 
      or double-bark thickness ('DBT')
    
    Returns
    -------
    dib : array
      diameter inside bark
    """
    dbh = np.asarray(dbh)
    dib_or_dbt = dib_or_dbt.upper()
    
    res = b1 + b2 * dbh**b3
    
    if dib_or_dbt == 'DBT':
        dib = dbh - res
    elif dib_or_dbt == 'DIB':
        dib = res
    
    return dib

In [23]:
def ccf_pn(dbh, r1, r2, r3, *args):
    dbh = np.asarray(dbh)
    ccf = np.empty_like(dbh)
    ccf[dbh >= 1.0] = r1 + r2*dbh[dbh >= 1.0] + r3*dbh[dbh >= 1.0]**2
    ccf[dbh < 1.0] = (r1 + r2 + r3) * dbh[dbh < 1.0]
    return ccf

def ccf_wc(dbh, r1, r2, r3, *args):
    dbh = np.asarray(dbh)
    ccf = np.empty_like(dbh)
    ccf[dbh >= 1.0] = r1 + r2*dbh[dbh >= 1.0] + r3*dbh[dbh >= 1.0]**2
    ccf[dbh < 1.0] = (r1 + r2 + r3) * dbh[dbh < 1.0]
    return ccf

def ccf_nc(dbh, r1, r2, r3, r4, r5):
    dbh = np.asarray(dbh)
    ccf = np.empty_like(dbh)
    ccf[dbh >= 1.0] = r1 + r2*dbh[dbh >= 1.0] + r3*dbh[dbh >= 1.0]**2
    ccf[dbh < 1.0] = r4*dbh[dbh < 1.0]**r5
    ccf[dbh <= 0.1] = 0.001
    return ccf

def ccf_so_453(dbh, r1, r2, r3, r4, r5):
    dbh = np.asarray(dbh)
    ccf = np.empty_like(dbh)
    ccf[dbh >= 1.0] = r1 + r2*dbh[dbh >= 1.0] + r3*dbh[dbh >= 1.0]**2
    ccf[dbh < 1.0] = r4*dbh[dbh < 1.0]**r5
    ccf[dbh <= 0.1] = 0.001
    return ccf

def ccf_so_454(dbh, r1, r2, r3, r4, r5):
    dbh = np.asarray(dbh)
    ccf = np.empty_like(dbh)
    ccf[dbh >= 1.0] = r1 + r2*dbh[dbh >= 1.0] + r3*dbh[dbh >= 1.0]**2
    ccf[dbh < 1.0] = (r1 + r2 + r3) * dbh[dbh < 1.0]
    ccf[dbh <= 0.1] = 0.001
    return ccf

def ccf_so_455(cw, r1, *args):
    cw = np.asarray(cw)
    ccf = r1*cw**2
    return ccf

def ccf_ca(cw, r1, *args):
    cw = np.asarray(cw)
    ccf = r1*cw**2
    return ccf

def ccf_ec_451(dbh, r1, r2, r3, r4, r5):
    dbh = np.asarray(dbh)
    ccf = np.empty_like(dbh)
    ccf[dbh >= 1.0] = r1 + r2*dbh[dbh >= 1.0] + r3*dbh[dbh >= 1.0]**2
    ccf[dbh < 1.0] = r4*dbh[dbh < 1.0]**r5
    ccf[dbh <= 0.1] = 0.001
    return ccf

def ccf_ec_454(dbh, r1, r2, r3, r4, r5):
    dbh = np.asarray(dbh)
    ccf = np.empty_like(dbh)
    ccf[dbh >= 1.0] = r1 + r2*dbh[dbh >= 1.0] + r3*dbh[dbh >= 1.0]**2
    ccf[dbh < 1.0] = (r1 + r2 + r3) * dbh[dbh < 1.0]
    ccf[dbh <= 0.1] = 0.001
    return ccf

def ccf_bm_451(dbh, r1, r2, r3, r4, r5):
    dbh = np.asarray(dbh)
    ccf = np.empty_like(dbh)
    ccf[dbh >= 1.0] = r1 + r2*dbh[dbh >= 1.0] + r3*dbh[dbh >= 1.0]**2
    ccf[dbh < 1.0] = r4*dbh[dbh < 1.0]**r5
    ccf[dbh <= 0.1] = 0.001
    return ccf

def ccf_bm_452(dbh, r1, r2, r3, r4, r5):
    dbh = np.asarray(dbh)
    ccf = np.empty_like(dbh)
    ccf[dbh >= 1.0] = r1 + r2*dbh[dbh >= 1.0] + r3*dbh[dbh >= 1.0]**2
    ccf[dbh < 1.0] = (r1 + r2 + r3) * dbh[dbh < 1.0]
    ccf[dbh <= 0.1] = 0.001
    return ccf

def ccf_ws_451(dbh, r1, r2, r3, r4, *args):
    dbh = np.asarray(dbh)
    ccf = np.empty_like(dbh)
    ccf[dbh >= 1.0] = ((r1 + r2*dbh[dbh >= 1.0])**2)*0.001803
    ccf[dbh < 1.0] = r3*dbh[dbh < 1.0]**r4
    ccf[dbh <= 0.1] = 0.001
    return ccf

def ccf_ws_452(cw, r1, *args):
    cw = np.asarray(cw)
    ccf = r1*cw**2
    return ccf

def ccf_ws_453(dbh, r1, r2, r3, r4, *args):
    dbh = np.asarray(dbh)
    ccf = np.empty_like(dbh)
    ccf[dbh >= 1.0] = r1 + r2*dbh[dbh >= 1.0] + r3*dbh[dbh >= 1.0]**2
    ccf[dbh < 1.0] = r4*dbh[dbh < 1.0]
    return ccf

def ccf_ws_454(dbh, r1, r2, r3, r4, r5):
    dbh = np.asarray(dbh)
    ccf = np.empty_like(dbh)
    ccf[dbh >= 10.0] = r1 + r2*dbh[dbh >= 10.0] + r3*dbh[dbh >= 10.0]**2
    ccf[dbh < 10.0] = r4*dbh[dbh < 10.0]**r5
    ccf[dbh <= 0.01] = 0.001
    return ccf

# for AK variant, source code for ccfcal.f indicates the documentation for the AK variant doesn't report the equations being used to calculate CCF
# we'll apply the PN variant CCF calcs for these trees on the Olympic Peninsula instead

ccf_eqns = {
    'PN': {'4.5.1': ccf_pn},
    'WC': {'4.5.1': ccf_wc},
    'NC': {'4.5.1': ccf_nc},
    'WS': {'4.5.1': ccf_ws_451,
           '4.5.2': ccf_ws_452,
           '4.5.3': ccf_ws_453,
           '4.5.4': ccf_ws_454,
          },
    'SO': {'4.5.1–4.5.3': ccf_so_453,
           '4.5.1&4.5.4': ccf_so_454,
           '4.5.5': ccf_so_455},
    'EC': {'4.5.1': ccf_ec_451,
           '4.5.1&4.5.4': ccf_ec_454},
    'BM': {'4.5.1': ccf_bm_451,
           '4.5.2': ccf_bm_452},
    'AK': {'4.5.1': ccf_pn,
           '4.5.2': ccf_pn},
    'CA': {'4.5.1': ccf_ca}
}

In [24]:
CCF_COEFS = '../data/raw/fvs_ccf_coefs.csv'
ccf_coefs = pd.read_csv(CCF_COEFS).set_index(['VARIANT', 'SPECIES'])
ccf_coefs.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 286 entries, ('PN', 'SF') to ('CA', 'OH')
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   R1        286 non-null    float64
 1   R2        225 non-null    float64
 2   R3        202 non-null    float64
 3   R4        123 non-null    float64
 4   R5        93 non-null     float64
 5   NOTES     286 non-null    object 
 6   EQN       286 non-null    object 
 7   CITATION  23 non-null     object 
dtypes: float64(5), object(3)
memory usage: 21.6+ KB


In [25]:
joined['CCF'] = np.nan
for (var, spp) in joined.groupby(by=['VARIANT', 'FVS_ALPHA']).groups:
    tmp_var = 'PN' if var == 'AK' else var
    try:
        coefs = ccf_coefs.loc[(tmp_var, spp)][['R1', 'R2', 'R3', 'R4', 'R5']].values
        eqn = ccf_coefs.loc[(tmp_var, spp)][['EQN']].iloc[0]
    except KeyError:
        print(var, tmp_var, spp)
        
    coefs = coefs[~pd.isnull(coefs)]
    ccf_eq = ccf_eqns[var][eqn]
    
    try:
        match = (joined.VARIANT == var)&(joined.FVS_ALPHA == spp)
        if (var == 'CA') or (var == 'SO' and eqn == '4.5.5') or (var == 'WS' and eqn == '4.5.2'):
            joined.loc[match, 'CCF'] = \
            ccf_eq(joined.loc[match, 'CW'], *coefs) * joined.loc[match, 'TPA']
        else:
            joined.loc[match, 'CCF'] = \
            ccf_eq(joined.loc[match, 'DBH'], *coefs) * joined.loc[match, 'TPA']
    except:
        print(var, spp, coefs, eqn)

pt_ccf = joined.groupby(by=['STANDID', 'PTINDEX'])['CCF'].sum().reset_index().rename({'CCF': 'PT_CCF'}, axis=1)
joined = joined.merge(pt_ccf, on=['STANDID', 'PTINDEX'], how='left')

In [26]:
joined['DIB'] = np.nan

for (var, spp) in joined.groupby(by=['VARIANT', 'FVS_ALPHA']).groups:
    match = (joined.VARIANT == var) & (joined.FVS_ALPHA == spp)
    try:
        pred, b1, b2, b3 = bark_eqs.loc[(var, spp)][['PREDICTS', 'B1', 'B2', 'B3']].iloc[0]
        joined.loc[match, 'DIB'] = calc_dib(joined.loc[match, 'DBH'].values, b1, b2, b3, pred)
    except:
        print(var, spp, pred, b1, b2, b3)

C:\Users\ddiaz\AppData\Local\Temp/ipykernel_31168/3857346117.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  pred, b1, b2, b3 = bark_eqs.loc[(var, spp)][['PREDICTS', 'B1', 'B2', 'B3']].iloc[0]


### Prune rows and columns
For modeling tree-level growth, we only need to keep the variables used in the growth or mortality equations. We also only want trees that have been measured more than once.

In [37]:
joined.set_index(['FIACOND', 'TREEID', 'TREEINDEX']).loc[remeasured_trees]

STANDID  TPA1000  DBH1000  \
FIACOND         TREEID TREEINDEX                                              
000601015502952 124    1          00062004050401015502952   131365     2200   
                       1          00062014060401015502952   245308     2900   
000601015506791 100    1          00062001050101015506791     6018     7000   
                       1          00062011060101015506791     8024     8200   
000601015519691 109    3000       00062008050801015519691        0    50000   
...                                                   ...      ...      ...   
005309071824592 104    2          00532015060409071824592    18650    22800   
                105    3          00532005050409071824592     3096    27300   
                       3          00532015060409071824592     3096    29000   
                106    4          00532005050409071824592     3096    34600   
                       4          00532015060409071824592     3096    37900   

                                  PTINDEX  YEAR  SPECIES FVS_ALPHA        DBH  \
FIACOND         TREEID TREEINDEX                                                
000601015502952 124    1                1  2004      117        SP   2.200000   
                       1                1  2014      117        SP   2.900000   
000601015506791 100    1                1  2001      631        TO   7.000000   
                       1                1  2011      631        TO   8.200000   
000601015519691 109    3000             1  2008      202        DF  50.000000   
...                                   ...   ...      ...       ...        ...   
005309071824592 104    2                1  2015      202        DF  22.799999   
                105    3                1  2005      202        DF  27.299999   
                       3                1  2015      202        DF  29.000000   
                106    4                1  2005      202        DF  34.599998   
                       4                1  2015      202        DF  37.900002   

                                         TPA     HT  ...  ELEVFT  \
FIACOND         TREEID TREEINDEX                     ...           
000601015502952 124    1          131.365463   11.0  ...    2700   
                       1          245.308456   13.0  ...    2700   
000601015506791 100    1            6.018046   41.0  ...    2500   
                       1            8.024061   47.0  ...    2500   
000601015519691 109    3000         0.000000  171.0  ...    2300   
...                                      ...    ...  ...     ...   
005309071824592 104    2           18.649506   82.0  ...    3100   
                105    3            3.096414   79.0  ...    3100   
                       3            3.096414   82.0  ...    3100   
                106    4            3.096414   88.0  ...    3100   
                       4            3.096414   87.0  ...    3100   

                                  SITE_SPECIES  SITE_INDEX  \
FIACOND         TREEID TREEINDEX                             
000601015502952 124    1                   202          67   
                       1                   202          67   
000601015506791 100    1                   202          75   
                       1                   202          75   
000601015519691 109    3000                202          50   
...                                        ...         ...   
005309071824592 104    2                   202          87   
                105    3                   202          87   
                       3                   202          87   
                106    4                   202          87   
                       4                   202          87   

                                  SITE_INDEX_BASE_AG VARIANT REL_SITE_INDEX  \
FIACOND         TREEID TREEINDEX                                              
000601015502952 124    1                          50      NC       0.360215   
                       1               

In [31]:
# keep only remeasured trees
remeasured_trees = [tuple(x) for x in joined.groupby(by=['FIACOND', 'TREEID', 'TREEINDEX'])['YEAR'].nunique().reset_index().query('YEAR > 1')[['FIACOND', 'TREEID', 'TREEINDEX']].values]
trees = joined.set_index(['FIACOND', 'TREEID', 'TREEINDEX']).loc[remeasured_trees].reset_index()
trees.head()

,FIACOND,TREEID,TREEINDEX,STANDID,TPA1000,DBH1000,PTINDEX,YEAR,SPECIES,FVS_ALPHA,...,ELEVFT,SITE_SPECIES,SITE_INDEX,SITE_INDEX_BASE_AG,VARIANT,REL_SITE_INDEX,RELHT,CCF,PT_CCF,DIB
0,000601015502952,124,1,00062004050401015502952,131365,2200,1,2004,117,SP,...,2700,202,67,50,NC,0.360215,0.220000,11.667723,103.112129,1.806960
1,000601015502952,124,1,00062014060401015502952,245308,2900,1,2014,117,SP,...,2700,202,67,50,NC,0.360215,0.464286,26.691695,39.018223,2.427370
2,000601015506791,100,1,00062001050101015506791,6018,7000,1,2001,631,TO,...,2500,202,75,50,NC,0.403226,0.650794,2.909484,200.806562,6.406540
3,000601015506791,100,1,00062011060101015506791,8024,8200,1,2011,631,TO,...,2500,202,75,50,NC,0.403226,0.587500,4.909101,434.803220,7.550788
4,000601015519691,109,3000,00062008050801015519691,0,50000,1,2008,202,DF,...,2300,202,50,50,NC,0.268817,1.449153,0.000000,427.491548,43.341005


In [38]:
min_year = [tuple(x) for x in trees.groupby(by=['FIACOND', 'TREEID', 'TREEINDEX'])['YEAR'].min().reset_index().values]
max_year = [tuple(x) for x in trees.groupby(by=['FIACOND', 'TREEID', 'TREEINDEX'])['YEAR'].max().reset_index().values]

In [62]:
first_meas = trees.set_index(['FIACOND', 'TREEID', 'TREEINDEX', 'YEAR']).loc[min_year].reset_index()
last_meas = trees.set_index(['FIACOND', 'TREEID', 'TREEINDEX', 'YEAR']).loc[max_year].reset_index().rename(
    {'DBH': 'NEXT_DBH', 'DIB': 'NEXT_DIB', 'YEAR': 'NEXT_YEAR', 'TREEVAL': 'NEXT_TREEVAL', 'HT': 'NEXT_HT', 'CR': 'NEXT_CR'}, axis=1)
meas = first_meas.merge(last_meas[['FIACOND', 'TREEID', 'TREEINDEX', 'SPECIES', 'NEXT_TREEVAL', 'NEXT_DBH', 'NEXT_DIB', 'NEXT_YEAR', 'NEXT_HT', 'NEXT_CR']], 
                        on=['FIACOND', 'TREEID', 'TREEINDEX', 'SPECIES'])
meas = meas.loc[meas['NEXT_YEAR'] > meas['YEAR']]
meas = meas.merge(fia_spp[['SPCD', 'COMMON_NAME']], left_on='SPECIES', right_on='SPCD', how='left')
meas['YEAR_INTERVAL'] = meas['NEXT_YEAR'] - meas['YEAR']
meas.head()

,FIACOND,TREEID,TREEINDEX,YEAR,STANDID,TPA1000,DBH1000,PTINDEX,SPECIES,FVS_ALPHA,...,DIB,NEXT_TREEVAL,NEXT_DBH,NEXT_DIB,NEXT_YEAR,NEXT_HT,NEXT_CR,SPCD,COMMON_NAME,YEAR_INTERVAL
0,000601015502952,124,1,2004,00062004050401015502952,131365,2200,1,117,SP,...,1.806960,1,2.900000,2.427370,2014,13.0,70,117.0,sugar pine,10
1,000601015506791,100,1,2001,00062001050101015506791,6018,7000,1,631,TO,...,6.406540,1,8.200000,7.550788,2011,47.0,30,631.0,tanoak,10
2,000601015519691,109,3000,2008,00062008050801015519691,0,50000,1,202,DF,...,43.341005,9,48.700001,42.225942,2018,165.0,10,202.0,Douglas-fir,10
3,000601015519691,111,2999,2008,00062008050801015519691,0,37000,1,202,DF,...,32.174989,9,42.900002,37.247070,2018,150.0,10,202.0,Douglas-fir,10
4,000601015535191,101,2,2009,00062009050901015535191,6018,14100,1,631,TO,...,13.176674,1,14.200000,13.272028,2019,55.0,35,631.0,tanoak,10


In [63]:
meas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66120 entries, 0 to 66119
Data columns (total 48 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   FIACOND             66120 non-null  object 
 1   TREEID              66120 non-null  int64  
 2   TREEINDEX           66120 non-null  int64  
 3   YEAR                66120 non-null  int64  
 4   STANDID             66120 non-null  object 
 5   TPA1000             66120 non-null  int64  
 6   DBH1000             66120 non-null  int64  
 7   PTINDEX             66120 non-null  int32  
 8   SPECIES             66120 non-null  int32  
 9   FVS_ALPHA           66120 non-null  object 
 10  DBH                 66120 non-null  float64
 11  TPA                 66120 non-null  float64
 12  HT                  66120 non-null  float64
 13  CR                  66120 non-null  int32  
 14  CW                  66120 non-null  float64
 15  PTBAL               66120 non-null  float64
 16  TREE

In [46]:
meas.groupby(by=['YEAR_INTERVAL'])['TREEID'].count()

YEAR_INTERVAL
5         2
8       103
9      1976
10    62551
11     1238
12      141
15        5
Name: TREEID, dtype: int64

In [56]:
(pd.pivot(meas.groupby(by=['VARIANT', 'COMMON_NAME'])['TREECN'].count().reset_index().rename({'TREECN': 'n'}, axis=1), 
          index=['COMMON_NAME'], columns=['VARIANT'], values=['n'])
 .fillna(0).astype(int)).head(50)

n                                     \
VARIANT                            AK    BM    CA    EC   NC    PN    SO   
COMMON_NAME                                                                
Alaska yellow-cedar                 0     0     0     9    0     4     0   
Brewer spruce                       0     0     1     0    0     0     0   
California black oak                0     0   243     0  361     0     0   
California buckeye                  0     0     9     0    0     0     0   
California juniper                  0     0     0     0    0     0     0   
California laurel                   0     0    21     0    0     0     0   
California live oak                 0     0    12     0    0     0     0   
California red fir                  0     0   100     0    3     0     0   
California white oak                0     0    13     0    0     0     0   
Coulter pine                        0     0     5     0    0     0     0   
Douglas-fir                         0  1838  1762  1711  963  6636   135   
Engelmann spruce                    0   257     0   106    0     1    37   
Jeffrey pine                        0     0   200     0    0     0     0   
Oregon ash                          0     0     3     0    0     0     0   
Oregon white oak                    0     0   149   156    0    30    64   
Pacific dogwood                     0     0    20     0    0     9     0   
Pacific madrone                     0     0   294     0  173     0     0   
Pacific silver fir                  1     0     0   344    0   197    11   
Pacific yew                         0     0    11     3    0     7     1   
Port-Orford-cedar                   0     0    38     0    0     0     0   
Shasta red fir                      0     0    24     0    0     0   144   
Sitka spruce                       13     0     0     0    0   219     0   
Washoe pine                         0     0     0     0    0     0     0   
bigcone Douglas-fir                 0     0     0     0    0     0     0   
bigleaf maple                       0     0    42    19    0   387     0   
bitter cherry                       0     0     0     0    0    22     1   
black cottonwood                    0     0     0     7    0    17     4   
blue oak                            0     0   137     0    0     0     0   
canyon live oak                     0     0   666     0    0     0     0   
cherry and plum spp.                0     0     0     8    0     0     0   
foxtail pine                        0     0     0     0    0     0     0   
giant chinkapin, golden chinkapin   0     0    72     4    0    23     1   
giant sequoia                       0     0     0     0    0     0     0   
grand fir                           0  1606     0   273    0    39    40   
gray or California foothill pine    0     0    60     0    0     0     0   
incense-cedar                       0     0   417     0   39    20   173   
interior live oak                   0     0    94     0    0     0     0   
knobcone pine                       0     0    25     0    0     0     0   
limber pine                         0     0     0     0    0     0     0   
lodgepole pine                      0   608    22   246    0    47  1366   
mountain hemlock                    0     0    22    71    0     5    88   
noble fir                           0     0     0    38    0    24     9   
other or unknown live tree          0     0     0     0    0    43     0   
paper birch                         0     0     0     9    0     7     0   
ponderosa pine                      0  3243   402   923   43     3  3826   
quaking aspen                       0     3     0     9    0     0     4   
red alder                          13     0    12    17    0   964     0   
singleleaf pinyon                   0     0     0     0    0     0     0   
subalpine fir                       0   114     0   129    0     9    13   
subalpine larch                     0     0     0    38    0     0     0   

       

In [64]:
KEEP_COLS = ['VARIANT', 'REGION', 'LOCATION', 'LAT', 'LON', 
             'FIACOND', 'CASEID', 'STANDID', 'PTINDEX', 'TREEID', 'TREEINDEX', 'TREECN', 
             'SPECIES', 'FVS_ALPHA', 'COMMON_NAME', 'YEAR', 'NEXT_YEAR',
             'TREEVAL', 'NEXT_TREEVAL', 'TPA', 
             'DBH', 'NEXT_DBH', 'DIB', 'NEXT_DIB', 
             'HT', 'RELHT', 'NEXT_HT', 'CR', 'NEXT_CR',
             # stand level metrics
             'TOPHT', 'BA', 'STAND_CCF', 'SLOPE', 'ASPECT', 'ELEVFT', 
             'SITE_SPECIES', 'SITE_INDEX_BASE_AG', 'SITE_INDEX', 'REL_SITE_INDEX', 
             # plot level metrics
             'PTBAL', 'PT_CCF']
RENAME = {'PTBAL': 'PT_BAL'}
meas[KEEP_COLS].rename(RENAME, axis=1).to_csv('../data/interim/remeasured_trees_for_training.csv', header=True, index=False)